### Load Dependencies

In [ ]:
%pip install opendatasets

### Imports

In [ ]:
import numpy as np
import opendatasets as od
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Subset
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from utils import ProgressBar

### Load Dataset

In [ ]:
# Download Kaggle dataset (Kaggle username and key is required)
# {"username":"christopherconroy","key":"1915e76943ae798bc236fb7c2de6d28d"}
od.download('https://www.kaggle.com/datasets/grassknoted/asl-alphabet')

In [ ]:
# Constants
TEST_DATA_DIR = 'asl-alphabet/asl_alphabet_test/asl_alphabet_test'
TRAIN_DATA_DIR = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
NUM_SAMPLES = 1000  # Number of dataset samples used (<= len(dataset))
TRAIN_SPLIT = 0.8  # Fraction of train data in train/valid split
SEED = 0  # Random seed for dataset shuffle and split
BATCH_SIZE = 100  # Minibatch size for training

In [ ]:
# Initialize dataset
dataset = torchvision.datasets.ImageFolder(
    root=TRAIN_DATA_DIR,
    transform=ToTensor()
)
num_inputs = np.array(dataset[0][0].numpy().shape).prod()
num_outputs = len(dataset.classes)

# Perform stratified split of dataset indicies
train_size = int((NUM_SAMPLES * TRAIN_SPLIT) // BATCH_SIZE) * BATCH_SIZE
test_size = NUM_SAMPLES - train_size
dataset_inds = list(range(len(dataset)))
train_inds, valid_inds = train_test_split(dataset_inds, train_size=train_size, 
        test_size=test_size, random_state=SEED, stratify=dataset.targets)

# Create training and validation subsets
train_set = Subset(dataset, train_inds)
valid_set = Subset(dataset, valid_inds)

# Initialize data loader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)

# Check for CUDA GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

### Training Implementation

In [ ]:
def train(data_loader, model, loss_func, optimizer):
    # Initialize parameters
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    total_loss = 0
    correct = 0

    # Set mode to training
    model.train()

    # Initialize progress bar
    progress = ProgressBar('Train Progress', len(data_loader))

    # Iterate through batches
    for images, labels in data_loader: 
        # Transfer images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        images = images.view(-1, num_inputs)
        
        # Forward pass 
        outputs = model(images)
        loss = loss_func(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Optimization
        optimizer.step()

        # Transfer outputs and labels to CPU
        outputs, labels = outputs.cpu(), labels.cpu()
        
        # Compute batch metrics
        total_loss += loss.item()
        pred = torch.max(outputs, 1)[1]
        correct += (pred == labels).sum().numpy()

        # Update progress
        progress.step()

    # Compute metrics for dataset
    total_loss /= num_batches
    accuracy = (correct / size) * 100

    return total_loss, accuracy

In [ ]:
def test(data_loader, model, loss_func):
    # Initialize parameters
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    total_loss = 0
    correct = 0

    # Set mode to evaluation
    model.eval()

    # Initialize progress bar
    progress = ProgressBar('Valid Progress', len(data_loader))

    # Iterate through batches
    with torch.no_grad():
        for images, labels in data_loader:
            # Transfer images and labels to GPU
            images, labels = images.to(device), labels.to(device)
            images = images.view(-1, num_inputs)

            # Forward pass
            outputs = model(images)
            loss = loss_func(outputs, labels)

            # Transfer outputs and labels to CPU
            outputs, labels = outputs.cpu(), labels.cpu()

            # Compute batch metrics
            total_loss += loss.item()
            pred = torch.max(outputs, 1)[1]
            correct += (pred == labels).sum().numpy()

            # Update progress
            progress.step()
            
    # Compute metrics for dataset
    total_loss /= num_batches
    accuracy = (correct / size) * 100

    return total_loss, accuracy

In [ ]:
def train_model(model, learning_rate, num_epochs, weight_decay=0):
    # Initialize training parameters
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Initialize metrics
    train_loss_list = []
    test_loss_list = []
    train_accuracy_list = []
    test_accuracy_list = []

    # Train model
    for epoch in range(num_epochs):
        # Train and evaluate model
        train_loss, train_accuracy = train(train_loader, model, loss_func, optimizer)
        valid_loss, valid_accuracy = test(valid_loader, model, loss_func)

        # Store epoch metrics
        train_loss_list.append(train_loss)
        train_accuracy_list.append(train_accuracy)
        test_loss_list.append(valid_loss)
        test_accuracy_list.append(valid_accuracy)

        # Output progress
        print('Epoch {} | Train Accuracy = {:.2f}% | Test Accuracy = {:.2f}%'
            .format(epoch + 1, train_accuracy, valid_accuracy))

    return (train_loss_list, train_accuracy_list), (test_loss_list, test_accuracy_list)

### Model

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.hidden1 = nn.Linear(num_inputs, 128)
        self.output = nn.Linear(128, num_outputs)

    def forward(self, x):
        x = self.hidden1(x)
        x = F.relu(x)
        x = self.output(x)
        return x

model1 = Net1()
model1.to(device)

In [ ]:
train_metrics1, test_metrics1 = train_model(model1, 0.01, 10)